In [7]:
import shared_functions
#下面用到什么定义就从shared_functions中载入
from shared_functions import read_from_files
import os
import pandas as pd
import numpy as np
import math
import sys
import time
import pickle
import json
import datetime
import random
import torch
import sklearn
from sklearn import *
from shared_functions import performance_assessment
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

import graphviz
import xgboost
# For imbalanced learning
import imblearn
import warnings
warnings.filterwarnings('ignore')
from shared_functions import training_loop
import math
import torch.nn as nn
from shared_functions import prepare_generators
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
DIR_INPUT='D:/Credit Card Fraud Detection/simulated-data-transformed/data/' 

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 938 ms
Wall time: 1.65 s
919767 transactions loaded, containing 8195 fraudulent transactions


In [3]:
from shared_functions import get_train_test_set, scaleData
# Set the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test
start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))
(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)
# By default, scales input data
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

In [4]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

Selected device is cuda


In [5]:
x_train = torch.FloatTensor(train_df[input_features].values)
x_valid = torch.FloatTensor(valid_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_valid = torch.FloatTensor(valid_df[output_feature].values)

In [6]:
# new Dataset: FraudDatasetUnsupervised, which only receives the descriptive features of the transaction x
# and returns it as both input and output.去掉标签的数据
class FraudDatasetUnsupervised(torch.utils.data.Dataset):
    
    def __init__(self, x,output=True):
        'Initialization'
        self.x = x
        self.output = output

    def __len__(self):
        'Returns the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample index
        item = self.x[index].to(DEVICE)
        if self.output:
            return item, item
        else:
            return item

In [8]:
training_set = FraudDatasetUnsupervised(x_train)
valid_set = FraudDatasetUnsupervised(x_valid)
training_generator,valid_generator = prepare_generators(training_set, valid_set, batch_size = 64)

In [9]:
def per_sample_mse(model, generator):
    
    model.eval()
    criterion = torch.nn.MSELoss(reduction="none")
    batch_losses = []
    
    for x_batch, y_batch in generator:
        # Forward pass
        y_pred = model(x_batch)
        # Compute Loss
        loss = criterion(y_pred.squeeze(), y_batch)
        loss_app = list(torch.mean(loss,axis=1).detach().cpu().numpy())
        batch_losses.extend(loss_app)
    
    return batch_losses

In [20]:
from tqdm import tqdm
import optuna

class GELU1(nn.Module):  # 激活函数含噪声参数
    def __init__(self, trial):
        super().__init__()
        self.sigma = trial.suggest_float("sigma", 0, 5)  # 1个可学习参数 sigma

    def forward(self, input):
        temp = input * (1 + torch.erf(input / torch.sqrt(torch.tensor(2.0)) / self.sigma)) / 2  # 前向传播
        return temp

class GELU2(nn.Module):  # 激活函数含噪声参数
    def __init__(self, trial):
        super().__init__()
        self.sigma = trial.suggest_float("sigma", 0, 5)  # 1个可学习参数 sigma

    def forward(self, input):
        temp = input * (1 + torch.erf(input / torch.sqrt(torch.tensor(2.0)) / self.sigma)) / 2 # 前向传播
        return temp

class GELU3(nn.Module):  # 激活函数含噪声参数
    def __init__(self, trial):
        super().__init__()
        self.sigma = trial.suggest_float("sigma", 0, 5)  # 1个可学习参数 sigma

    def forward(self, input):
        temp = input * (1 + torch.erf(input / torch.sqrt(torch.tensor(2.0)) / self.sigma)) / 2  # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.gelu1 = GELU1(trial)  
            self.gelu2 = GELU2(trial)
            self.gelu3 = GELU3(trial)
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.gelu1(hidden)
            
            code = self.fc2(hidden)
            code = self.gelu2(code)
 
            hidden = self.fc3(code)
            hidden = self.gelu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

In [21]:
def objective(trial):
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    weight_auc = 0.5
    weight_card_precision = 0.5
    target_value = weight_auc * auc_roc + weight_card_precision * card_precision
    print(auc_roc)
    print(card_precision)
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)

[I 2024-12-02 11:01:53,964] A new study created in memory with name: no-name-326d89c3-ced3-4bb3-914e-d9656f74f10f



Epoch 0: train loss: 0.4591187751116534
valid loss: 0.13595747656998086

Epoch 1: train loss: 0.09128768595530301
valid loss: 0.041978733897046316

Epoch 2: train loss: 0.033257889682444856
valid loss: 0.02258824352378565

Epoch 3: train loss: 0.01836601363754979
valid loss: 0.011185668735596977

Epoch 4: train loss: 0.009021155097001868
valid loss: 0.005699780491776153

Epoch 5: train loss: 0.005568520039102743
valid loss: 0.004239475641201144

Epoch 6: train loss: 0.004018128606941316
valid loss: 0.0028906637053336195

Epoch 7: train loss: 0.0027859917274020954
valid loss: 0.0020419995316375513

Epoch 8: train loss: 0.0019749947943783455
valid loss: 0.0016298140868636183

Epoch 9: train loss: 0.0016142471121284882
valid loss: 0.0012737840398461583

Epoch 10: train loss: 0.0013945126648610333
valid loss: 0.0011629239831926414

Epoch 11: train loss: 0.0012221333398436949
valid loss: 0.001019301015439652

Epoch 12: train loss: 0.0010954837048398223
valid loss: 0.0009127289810932948

Ep

[I 2024-12-02 11:13:19,270] Trial 0 finished with value: 0.533 and parameters: {'sigma': 1.048107223753307}. Best is trial 0 with value: 0.533.


0.843
0.223

Epoch 0: train loss: 0.5214429405690607
valid loss: 0.20830944890532987

Epoch 1: train loss: 0.12821402439908133
valid loss: 0.0730285472923615

Epoch 2: train loss: 0.05856339439187007
valid loss: 0.03589558845660726

Epoch 3: train loss: 0.026672678881184148
valid loss: 0.01699234146950512

Epoch 4: train loss: 0.013283016022568992
valid loss: 0.008389742560481113

Epoch 5: train loss: 0.007047137070801572
valid loss: 0.004899249537313571

Epoch 6: train loss: 0.004987541702464686
valid loss: 0.0037018115357419505

Epoch 7: train loss: 0.0039275977367679476
valid loss: 0.0029732841089917497

Epoch 8: train loss: 0.0030108211698040835
valid loss: 0.0021368078587836294

Epoch 9: train loss: 0.002053498091538086
valid loss: 0.0013822413443414687

Epoch 10: train loss: 0.0013668650568593793
valid loss: 0.001040389523472474

Epoch 11: train loss: 0.0010742124041351428
valid loss: 0.00089355561258195

Epoch 12: train loss: 0.0009200939547246497
valid loss: 0.00070669067413813

[I 2024-12-02 11:24:12,287] Trial 1 finished with value: 0.545 and parameters: {'sigma': 1.828143286341829}. Best is trial 1 with value: 0.545.


...

0.849
0.231

Epoch 0: train loss: 0.4378193155087773
valid loss: 0.11987121287997954

Epoch 1: train loss: 0.07970211919696278
valid loss: 0.0412274345857361

Epoch 2: train loss: 0.032466912872726045
valid loss: 0.020560026907774268

Epoch 3: train loss: 0.015776371267707895
valid loss: 0.009381608338263191

Epoch 4: train loss: 0.008576758297153144
valid loss: 0.0065207480521658895

Epoch 5: train loss: 0.006905970887510715
valid loss: 0.005707721777331552

Epoch 6: train loss: 0.005950153335202379
valid loss: 0.00482247008819927

Epoch 7: train loss: 0.005151057186719907
valid loss: 0.004127699467531645

Epoch 8: train loss: 0.004351065823110266
valid loss: 0.003381280110299058

Epoch 9: train loss: 0.0034620272968165634
valid loss: 0.002653161818008931

Epoch 10: train loss: 0.002647881085154588
valid loss: 0.0021375925819608356

Epoch 11: train loss: 0.0021620977938844755
valid loss: 0.0020068596409827145

Epoch 12: train loss: 0.0019033184185660629
valid loss: 0.00184691493047014

[I 2024-12-02 12:38:37,468] Trial 8 finished with value: 0.5255 and parameters: {'sigma': 0.5149413045557832}. Best is trial 1 with value: 0.545.


0.85
0.201

Epoch 0: train loss: 0.528051650333245
valid loss: 0.204465729201752

Epoch 1: train loss: 0.14023126247935497
valid loss: 0.07809339370476744

Epoch 2: train loss: 0.05939550796099876
valid loss: 0.03957885546721722

Epoch 3: train loss: 0.03051758619082333
valid loss: 0.02113264371218577

Epoch 4: train loss: 0.018911464515998674
valid loss: 0.013418330575843327

Epoch 5: train loss: 0.01114759503073107
valid loss: 0.007049680780110463

Epoch 6: train loss: 0.006336631405614245
valid loss: 0.004454353900468537

Epoch 7: train loss: 0.004720151880052632
valid loss: 0.0035231514594675412

Epoch 8: train loss: 0.003907875059841176
valid loss: 0.0029299413509249607

Epoch 9: train loss: 0.0031983721810998483
valid loss: 0.0023692745598328244

Epoch 10: train loss: 0.0023645738053605686
valid loss: 0.001655215266684854

Epoch 11: train loss: 0.0015570030539999067
valid loss: 0.001089112843702359

Epoch 12: train loss: 0.0011352117127019033
valid loss: 0.0008474322624982951

Ep

[I 2024-12-02 12:49:18,576] Trial 9 finished with value: 0.539 and parameters: {'sigma': 2.1575385190845013}. Best is trial 1 with value: 0.545.


0.854
0.224


In [22]:
0.856
0.234

0.234

In [23]:
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: GELU1={sigma1}, GELU2={sigma2}, GELU3={sigma3}")

Best trial parameters:
{'sigma': 1.828143286341829}
Optimal sigma values: GELU1=1.828143286341829, GELU2=1.828143286341829, GELU3=1.828143286341829


In [29]:
best_params = study.best_trial.params

# 使用最佳 sigma 值初始化模型（不依赖 trial）
best_model = SimpleAutoencoder(len(input_features), 100, 20, trial=None).to(DEVICE)

# 手动设置 GELU 层的最佳 sigma 值
best_model.gelu1.sigma = torch.tensor(best_params['sigma'], device=DEVICE)
best_model.gelu2.sigma = torch.tensor(best_params['sigma'], device=DEVICE)
best_model.gelu3.sigma = torch.tensor(best_params['sigma'], device=DEVICE)

# 计算每个样本的 MSE 损失
losses = per_sample_mse(best_model, valid_generator)

# 更新验证集 DataFrame
predictions_df = valid_df.copy()
predictions_df['predictions'] = losses

# 评估性能指标
performances = performance_assessment(predictions_df, top_k_list=[100])

AttributeError: 'NoneType' object has no attribute 'suggest_float'